<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/05_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 75.5 MB/s 


In [12]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [13]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [14]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [15]:
classes, class_weight = get_class_weight()

In [16]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [20]:
target_shape = (32, 48, 1)

rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL, trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL, trainable=True)


rgb_features = rgb_hub_module(rgb_input)
rgb_layer = keras.layers.Reshape(target_shape=target_shape)(rgb_features)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)
xyz_layer = keras.layers.Reshape(target_shape=target_shape)(xyz_features)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)
hsv_layer = keras.layers.Reshape(target_shape=target_shape)(hsv_features)

block = keras.layers.Concatenate()([rgb_layer, xyz_layer, hsv_layer])

conv1 = keras.layers.Conv2D(filters=10,
                            kernel_size=(2,2),
                            strides=(2,3),
                            padding='same',
                            activation='relu')(block)

conv2 = keras.layers.Conv2D(filters=20,
                            kernel_size=(2,2),
                            strides=(2,2),
                            activation='relu')(conv1)

avg_pool = keras.layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = keras.layers.Conv2D(filters=20,
                            kernel_size=(1,1),
                            activation='relu')(avg_pool)

flatten = keras.layers.Flatten()(conv3)

output = keras.layers.Dense(N_CLASSES, activation='softmax')(flatten)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [21]:
model._name = 'combined-model-v1'

In [23]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [24]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_auc', 
                                              verbose=1,
                                              patience=5,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+".h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [26]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15


ResourceExhaustedError: ignored

In [ ]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.1.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.1.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.1.h5

In [ ]:
history = model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=20,
                        class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 419s 805ms/step - loss: 1.6773 - auc: 0.9835 - accuracy: 0.9476
Epoch 22/30
519/519 [==============================] - 419s 804ms/step - loss: 1.6029 - auc: 0.9848 - accuracy: 0.9497
Epoch 23/30
519/519 [==============================] - 419s 804ms/step - loss: 1.4878 - auc: 0.9859 - accuracy: 0.9527
Epoch 24/30
519/519 [==============================] - 419s 804ms/step - loss: 1.4111 - auc: 0.9869 - accuracy: 0.9562
Epoch 25/30
519/519 [==============================] - 419s 805ms/step - loss: 1.3436 - auc: 0.9876 - accuracy: 0.9578
Epoch 26/30
519/519 [==============================] - 419s 804ms/step - loss: 1.2742 - auc: 0.9880 - accuracy: 0.9603
Epoch 27/30
519/519 [==============================] - 419s 805ms/step - loss: 1.2097 - auc: 0.9892 - accuracy: 0.9615
Epoch 28/30
519/519 [==============================] - 419s 805ms/step - loss: 1.1573 - auc: 0.9894 - accuracy: 0.9654
Epoch 29/30
519/519 [===========================

In [ ]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.1.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.1.h5

In [ ]:
history = model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=35,
                        initial_epoch=30,
                        class_weight=class_weight)

Epoch 31/35
519/519 [==============================] - 420s 806ms/step - loss: 1.0314 - auc: 0.9908 - accuracy: 0.9685
Epoch 32/35
519/519 [==============================] - 419s 804ms/step - loss: 0.9957 - auc: 0.9910 - accuracy: 0.9689
Epoch 33/35
519/519 [==============================] - 419s 805ms/step - loss: 0.9585 - auc: 0.9913 - accuracy: 0.9704
Epoch 34/35
519/519 [==============================] - 419s 804ms/step - loss: 0.9375 - auc: 0.9919 - accuracy: 0.9712
Epoch 35/35
519/519 [==============================] - 418s 804ms/step - loss: 0.9046 - auc: 0.9921 - accuracy: 0.9721


In [ ]:
!cp ./logs/checkpoints/combined-model-35.h5 ./drive/MyDrive/checkpoints/combined-model-35.1.h5